In [56]:
using DataFrames
using BenchmarkTools
using StatsBase

Implement the logic for reading mgf files.<br>
Try to reproduce the Python function as close as possible

In [36]:
function loadmgf(fname)
    FIELDS = ["TITLE=", "RTINSECONDS=", "PEPMASS=", "CHARGE=", "SCANS="]
    
    function format_precursor!(spectrum)
        if occursin( " ", spectrum["PEPMASS"] )
            spectrum["PEPMASS"] = map( x -> tryparse(Float64, x), split( spectrum["PEPMASS"] , " " ) )
        else
            spectrum["PEPMASS"] = [ tryparse(Float64, spectrum["PEPMASS"] ) ]
        end
        
        local polarity_multiplier = 1
        if spectrum["CHARGE"][end] == '-'
            polarity_multiplier = -1
        end
        
        if isnumeric( spectrum["CHARGE"][end] ) == false
            spectrum["CHARGE"] = polarity_multiplier * tryparse(Int32, spectrum["CHARGE"][1:end-1] )
        else
            spectrum["CHARGE"] = tryparse(Int32, spectrum["CHARGE"])
        end
        return true
    end
    
    function msdata_to_df!(spectrum)
        spectrum["ms_data"] = DataFrame( spectrum["ms_data"] )
        return true
    end
    
    local spectrum
    spectra_array = []
    open(fname) do fh
        state = false
        for line in eachline(fh)
            
            if length(line) > 0 && isnumeric( line[1] ) && state == true
                num_line = map( x -> tryparse(Float64, x), split(line, " ") )
                push!(
                    spectrum["ms_data"]["m/z"],
                    num_line[1]
                )
                push!(
                    spectrum["ms_data"]["Intensity"],
                    num_line[2]
                )
            elseif occursin("BEGIN IONS", line)
                spectrum = Dict{String,Any}( [ "ms_data" => Dict(
                            [
                                "m/z"=> Array{Float64}(undef, 0),
                                "Intensity"=> Array{Float64}(undef, 0)
                            ]
                        ) ] )
                state = true
                
            elseif occursin("END IONS", line)
                #println(spectrum)
                if length( spectrum["ms_data"] ) > 0
                    format_precursor!(spectrum)
                    msdata_to_df!(spectrum)
                    push!(spectra_array, spectrum)
                end
                state = false
            else
                for field_name in FIELDS
                    if occursin(field_name, line) && state == true
                        spectrum[ field_name[1:end-1] ] = split(line, field_name)[2]
                    end
                end
            end
        end
    end
    spectra_array
end

loadmgf (generic function with 2 methods)

In [37]:
fname = "Yeast_1000spectra.mgf"

"Yeast_1000spectra.mgf"

In [38]:
res = loadmgf(fname)
length(res)

1000

In [39]:
res[501]

Dict{String, Any} with 5 entries:
  "TITLE"       => "Fusion_180828_07.13836.13836.2 File:\"Fusion_180828_07.raw\…
  "RTINSECONDS" => "2463.272073"
  "CHARGE"      => 2
  "ms_data"     => 100×2 DataFrame…
  "PEPMASS"     => [525.756, 4.28586e6]

In [40]:
res[501]["ms_data"]

,Intensity,m/z
,Float64,Float64
1,262.677,172.973
2,117.925,175.068
3,170.186,189.044
4,120.847,201.158
5,163.982,221.101
6,66.9147,221.977
7,131.333,249.024
8,117.201,249.966
9,107.119,259.098


In [97]:
@benchmark loadmgf(fname) samples=200

BenchmarkTools.Trial: 55 samples with 1 evaluation.
 Range (min … max):  85.196 ms … 113.032 ms  ┊ GC (min … max): 6.36% … 10.74%
 Time  (median):     88.589 ms               ┊ GC (median):    6.85%
 Time  (mean ± σ):   91.118 ms ±   5.716 ms  ┊ GC (mean ± σ):  7.33% ±  1.86%

  █▄▄█ █ █    ▁                  █ ▁▄             ▁          ▁  
  ████▆█▆█▁▆▁▆█▁▁▁▁▁▁▁▆▁▆▆▆▁▁▆▁▆▆█▆██▆▁▆▁▁▁▆▁▆▁▁▁▁█▁▁▁▁▁▁▁▁▁▆█ ▁
  85.2 ms         Histogram: frequency by time          102 ms <

 Memory estimate: 48.95 MiB, allocs estimate: 658670.

Let's now check each spectrum for known mass differences

Monoisotopic masses of amino acids:

In [100]:
AA_DELTAS = Dict(
    'G'=> 57.02147, 'A'=> 71.03712, 'S'=> 87.03203, 'P'=> 97.05277, 'V'=> 99.06842,
    'T'=> 101.04768, "Ccam"=> 160.03065, "Cmes"=> 148.996912, "I/L"=> 113.08407,
    'N'=> 114.04293, 'D'=> 115.02695, 'Q'=> 128.05858, 'K'=> 128.09497, 'E'=> 129.0426,
    'M'=> 131.04049, "Mox"=> 147.0354, 'H'=> 137.05891, 'F'=> 147.06842, 'R'=> 156.10112,
    'Y'=> 163.06333, 'W'=> 186.07932
)

Dict{Any, Float64} with 21 entries:
  "Mox"  => 147.035
  'E'    => 129.043
  'D'    => 115.027
  'A'    => 71.0371
  "I/L"  => 113.084
  'R'    => 156.101
  'G'    => 57.0215
  'N'    => 114.043
  'Q'    => 128.059
  'P'    => 97.0528
  'K'    => 128.095
  'M'    => 131.04
  'F'    => 147.068
  'H'    => 137.059
  'W'    => 186.079
  'S'    => 87.032
  'T'    => 101.048
  "Ccam" => 160.031
  "Cmes" => 148.997
  'Y'    => 163.063
  'V'    => 99.0684

In [43]:
#Flatten the values from the dictionary
single_res_Δ = collect( values(AA_DELTAS) )
println( typeof(single_res_Δ) )
#Add doubly-charged and triply-charged mass Deltas (simply divide by 2 and 3)
single_res_Δ = vcat( single_res_Δ / 3, single_res_Δ / 2, single_res_Δ )
println( length( single_res_Δ ) )
single_res_Δ[1:5]

Vector{Float64}
63


5-element Vector{Float64}:
 42.686193333333335
 23.67904
 49.0118
 33.68256
 32.350923333333334

Now take the spectra one-by-one, find pairwise mass differences and match them to the list.<br>
Reproduce the logic from the Python function as closely as possible:<br>
* Calculate pairwise absolute differences between the 
* Subtract the experimental mass Deltas from the theoretical
* Calculate relative difference
* Select the cases with the relative difference lower than threshold (matches)
* Summarize and report the matches

In [44]:
function matches(spectra, masses_to_match, rel_tolerance)
    res_dict = Dict([
        ( "Spectrum_idx", Vector{Int64}() ),
        ( "Exp_idx", Vector{Int64}() ),
        ( "Library_idx", Vector{Int64}() ),
        ( "Rel_error", Vector{Float64}() )
    ])
    min_theo_val = minimum(masses_to_match) * (1 - rel_tolerance)
   for (idx, s) in enumerate(spectra)
        exp_Δs = filter(
            x -> x > min_theo_val,
            s["ms_data"][!, "m/z"] .- s["ms_data"][!, "m/z"]'
        )
        rel_deltas_matrix = map(
            abs,
            (masses_to_match .- exp_Δs')
        ) * 2 ./ (masses_to_match .+ exp_Δs')
        matching_inds = findall(x -> isless(x, rel_tolerance), rel_deltas_matrix)
        num_matches = size(matching_inds)[1]
        if num_matches > 0
            append!( res_dict["Spectrum_idx"], fill(idx, num_matches) )
            append!( res_dict["Library_idx"], map(x -> x[1], matching_inds) )
            append!( res_dict["Exp_idx"], map(x -> x[2], matching_inds) )
            append!( res_dict["Rel_error"], rel_deltas_matrix[matching_inds] )
        end
    end
    DataFrame(res_dict)
end

matches (generic function with 1 method)

In [45]:
m = matches(res, single_res_Δ, 1e-5)
size(m)

(1424, 4)

In [105]:
@benchmark matches(res, single_res_Δ, 1e-5) samples=50

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.025 s …   1.087 s  ┊ GC (min … max): 16.06% … 15.39%
 Time  (median):     1.045 s              ┊ GC (median):    15.93%
 Time  (mean ± σ):   1.050 s ± 25.638 ms  ┊ GC (mean ± σ):  15.80% ±  0.30%

  █  █             █                 █                    █  
  █▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.03 s         Histogram: frequency by time        1.09 s <

 Memory estimate: 5.87 GiB, allocs estimate: 38543.

Use of the dot macro to fuse several matrix operations into one

In [47]:
function matchesfuse(spectra, masses_to_match, rel_tolerance)
    res_dict = Dict([
        ( "Spectrum_idx", Vector{Int64}() ),
        ( "Exp_idx", Vector{Int64}() ),
        ( "Library_idx", Vector{Int64}() ),
        ( "Rel_error", Vector{Float64}() )
    ])
    min_theo_val = minimum(masses_to_match) * (1 - rel_tolerance)
   for (idx, s) in enumerate(spectra)
        exp_Δs = filter(
            x -> x > min_theo_val,
            s["ms_data"][!, "m/z"] .- s["ms_data"][!, "m/z"]'
        )
        #Fuse the vectorized operations using the dot macro
        rel_deltas_matrix = @. map(abs, (masses_to_match - exp_Δs') ) * 2 / (masses_to_match + exp_Δs')
        matching_inds = findall(x -> isless(x, rel_tolerance), rel_deltas_matrix)
        num_matches = size(matching_inds)[1]
        if num_matches > 0
            append!( res_dict["Spectrum_idx"], fill(idx, num_matches) )
            append!( res_dict["Library_idx"], map(x -> x[1], matching_inds) )
            append!( res_dict["Exp_idx"], map(x -> x[2], matching_inds) )
            append!( res_dict["Rel_error"], rel_deltas_matrix[matching_inds] )
        end
    end
    DataFrame(res_dict)
end

matchesfuse (generic function with 1 method)

In [48]:
m = matchesfuse(res, single_res_Δ, 1e-5)
size(m)

(1424, 4)

In [49]:
m[ m.Spectrum_idx .== 2 , :]

,Exp_idx,Library_idx,Rel_error,Spectrum_idx
,Int64,Int64,Float64,Int64
1,693,50,5.0423e-7,2
2,1368,39,7.61781e-6,2
3,2200,56,4.01173e-6,2
4,4006,11,8.89701e-6,2
5,4085,38,8.90187e-6,2
6,4085,51,8.81418e-6,2


In [106]:
@benchmark matchesfuse(res, single_res_Δ, 1e-5) samples=50

BenchmarkTools.Trial: 13 samples with 1 evaluation.
 Range (min … max):  402.164 ms … 433.757 ms  ┊ GC (min … max): 8.96% … 10.53%
 Time  (median):     406.995 ms               ┊ GC (median):    9.08%
 Time  (mean ± σ):   410.163 ms ±   9.430 ms  ┊ GC (mean ± σ):  9.30% ±  0.68%

  ▁ █ ▁▁ ▁ ▁▁   ▁▁     ▁                        ▁             ▁  
  █▁█▁██▁█▁██▁▁▁██▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  402 ms           Histogram: frequency by time          434 ms <

 Memory estimate: 1.57 GiB, allocs estimate: 38549.

Create a list of random peptide sequences

In [51]:
aa_curated = [
    'G', 'A', 'S', 'P', 'V', 'T', 'N', 'D', 'Q', 'K', 'E', 'M', 'H', 'F', 'R', 'Y', 'W'
]

17-element Vector{Char}:
 'G': ASCII/Unicode U+0047 (category Lu: Letter, uppercase)
 'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)
 'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)
 'P': ASCII/Unicode U+0050 (category Lu: Letter, uppercase)
 'V': ASCII/Unicode U+0056 (category Lu: Letter, uppercase)
 'T': ASCII/Unicode U+0054 (category Lu: Letter, uppercase)
 'N': ASCII/Unicode U+004E (category Lu: Letter, uppercase)
 'D': ASCII/Unicode U+0044 (category Lu: Letter, uppercase)
 'Q': ASCII/Unicode U+0051 (category Lu: Letter, uppercase)
 'K': ASCII/Unicode U+004B (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'M': ASCII/Unicode U+004D (category Lu: Letter, uppercase)
 'H': ASCII/Unicode U+0048 (category Lu: Letter, uppercase)
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'R': ASCII/Unicode U+0052 (category Lu: Letter, uppercase)
 'Y': ASCII/Unicode U+0059 (category Lu: Letter, uppercase)
 'W': ASCII/Uni

In [57]:
sample(aa_curated)

'H': ASCII/Unicode U+0048 (category Lu: Letter, uppercase)

In [59]:
a = 'H'
a *= 'A'
a

"HA"

In [60]:
a = ""
for i = 1:20
    a *= sample(aa_curated)
end
a

"HEVDKHKSATGNQDVMNDDR"

In [62]:
length(a)

20

In [89]:
sequences_list = Array([])
for i = 1:10000
    a = ""
    for j = 1:20
        a *= sample(aa_curated)
    end
    append!(sequences_list, [a])
end
length(sequences_list)

10000

In [90]:
sequences_list[1:5]

5-element Vector{Any}:
 "SSFPNDQQAGHHPPWVNAGP"
 "FEMWNFRHMDPGFFHVETWK"
 "TVRPGSQQNRYKHAHGVDRQ"
 "KAGDYPGYSGDQQSSVYYEH"
 "SRFNHNHVMSKMADQNVANH"

Create a function for calculating peptides masses using for loops

In [91]:
for i in split("ERPPPGVMGDRSYPVRFVDP", "")
    println(AA_DELTAS[i])
end

129.0426
156.10112
97.05277
97.05277
97.05277
57.02147
99.06842
131.04049
57.02147
115.02695
156.10112
87.03203
163.06333
97.05277
99.06842
156.10112
147.06842
99.06842
115.02695
97.05277


In [111]:
function calculate_masses_loop(sequences_list)
    masses = Array{Float64}([])
    for i in sequences_list
        m = 18.010565
        for j in i
            m += AA_DELTAS[j]
        end
        append!(masses, [m])
    end
    masses
end

calculate_masses_loop (generic function with 1 method)

In [112]:
l = calculate_masses_loop(sequences_list)
length(l)

10000

In [113]:
l[1:5]

5-element Vector{Float64}:
 2141.9668349999997
 2640.183195
 2333.1850449999997
 2249.950235
 2336.0651750000006

In [116]:
@benchmark calculate_masses_loop(sequences_list) samples=200

BenchmarkTools.Trial: 190 samples with 1 evaluation.
 Range (min … max):  23.294 ms … 43.511 ms  ┊ GC (min … max): 0.00% … 8.12%
 Time  (median):     25.903 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.314 ms ±  1.999 ms  ┊ GC (mean ± σ):  3.33% ± 4.76%

            ▅▂          █▅▂        ▁                           
  ▃▃▁▁▁▃▃▃▆▆██▃▃▁▁▁▃▃▃▄▄███▄▃▁▁▃▃▅▅█▃▄▃▄▁▁▁▁▃▁▃▄▅▆▆▄▃▁▃▃▁▃▁▁▃ ▃
  23.3 ms         Histogram: frequency by time        29.9 ms <

 Memory estimate: 13.14 MiB, allocs estimate: 610010.